In [1]:
import pandas as pd
import snowflake.snowpark.functions as F
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session

In [2]:
session = Session.builder.configs(SnowflakeLoginOptions()).getOrCreate()

SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 


In [3]:
session.sql("CREATE DATABASE IF NOT EXISTS SNOWPARK;").collect()
session.sql("CREATE SCHEMA IF NOT EXISTS SNOWPARK.TITANIC;").collect()

[Row(status='TITANIC already exists, statement succeeded.')]

In [4]:
session.sql("CREATE STAGE if not exists ML_DATA").collect()

[Row(status='ML_DATA already exists, statement succeeded.')]

In [5]:
titanic = pd.read_csv(
    "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv"
)
titanic.columns = [c.upper() for c in titanic.columns]
titanic.to_csv("titanic.csv", index=False)

In [6]:
session.file.put("titanic.csv", "@ml_data", overwrite=True)

[PutResult(source='titanic.csv', target='titanic.csv.gz', source_size=57018, target_size=6528, source_compression='NONE', target_compression='GZIP', status='UPLOADED', message='')]

In [7]:
# Check the state of the warehouse
warehouse_state = session.sql("SHOW WAREHOUSES LIKE 'compute_wh'").collect()
print(warehouse_state)

session.sql("ALTER WAREHOUSE compute_wh RESUME").collect()



[Row(name='COMPUTE_WH', state='SUSPENDED', type='STANDARD', size='X-Small', min_cluster_count=1, max_cluster_count=1, started_clusters=0, running=0, queued=0, is_default='Y', is_current='Y', auto_suspend=600, auto_resume='true', available='', provisioning='', quiescing='', other='', created_on=datetime.datetime(2024, 2, 9, 6, 10, 45, 736000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), resumed_on=datetime.datetime(2024, 2, 26, 16, 6, 3, 488000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), updated_on=datetime.datetime(2024, 2, 26, 16, 6, 3, 488000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), owner='SYSADMIN', comment='', enable_query_acceleration='false', query_acceleration_max_scale_factor=8, resource_monitor='null', actives=0, pendings=0, failed=0, suspended=1, uuid='63687428', scaling_policy='STANDARD', budget=None, owner_role_type='ROLE')]


[Row(status='Statement executed successfully.')]

In [8]:
# Create a Snowpark DataFrame that is configured to load data from the CSV file
titanic_df = (
    session.read.option("infer_schema", True)
    .option("PARSE_HEADER", True)
    .csv("@ml_data/titanic.csv")
)
titanic_df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SURVIVED"  |"PCLASS"  |"SEX"   |"AGE"  |"SIBSP"  |"PARCH"  |"FARE"   |"EMBARKED"  |"CLASS"  |"WHO"  |"ADULT_MALE"  |"DECK"  |"EMBARK_TOWN"  |"ALIVE"  |"ALONE"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0           |3         |male    |22.00  |1        |0        |7.2500   |S           |Third    |man    |True          |NULL    |Southampton    |False    |False    |
|1           |1         |female  |38.00  |1        |0        |71.2833  |C           |First    |woman  |False         |C       |Cherbourg      |True     |False    |
|1           |3         |female  |26.00  |0        |0        |7.9250   |S           |Third    |woman  |False         |NULL    |Southampton    |True     |True     |
|1           |1 

In [9]:
def fix_values(column):
    return F.upper(F.regexp_replace(F.col(column), "[^a-zA-Z0-9]+", "_"))


for col in ["SEX", "EMBARKED", "CLASS", "WHO", "EMBARK_TOWN"]:
    titanic_df = titanic_df.with_column(col, fix_values(col))

titanic_df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SURVIVED"  |"PCLASS"  |"AGE"  |"SIBSP"  |"PARCH"  |"FARE"   |"ADULT_MALE"  |"DECK"  |"ALIVE"  |"ALONE"  |"SEX"   |"EMBARKED"  |"CLASS"  |"WHO"  |"EMBARK_TOWN"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0           |3         |22.00  |1        |0        |7.2500   |True          |NULL    |False    |False    |MALE    |S           |THIRD    |MAN    |SOUTHAMPTON    |
|1           |1         |38.00  |1        |0        |71.2833  |False         |C       |True     |False    |FEMALE  |C           |FIRST    |WOMAN  |CHERBOURG      |
|1           |3         |26.00  |0        |0        |7.9250   |False         |NULL    |True     |True     |FEMALE  |S           |THIRD    |WOMAN  |SOUTHAMPTON    |
|1           |1 

In [10]:
titanic_df.write.mode("overwrite").save_as_table("titanic")